# Workspace creation (minimal)

*Michele Pinamonti - INFN Sezione di Trieste*

Simplified version of `create_workspace`

Import ROOT and glob

In [ ]:
import ROOT
import os

Create output directories

In [ ]:
!mkdir -p ../ws
!mkdir -p ../xml

Create the measurement object, set prefix for outputs, set parameter of insterest, plus a number of needed settings

In [ ]:
meas = ROOT.RooStats.HistFactory.Measurement('ICTPws_minimal', 'ICTPws_minimal')
meas.SetOutputFilePrefix('../ws/ICTPws_minimal')

meas.SetPOI('mu_ttH')

meas.AddConstantParam('Lumi')
meas.SetLumi(1.0)
meas.SetLumiRelErr(0.0)
meas.SetExportOnly(True)

Create a channel - the signal region

In [ ]:
chan_sr = ROOT.RooStats.HistFactory.Channel( "ljets_Mbb_ge6jge4b" )

Store the input file name

In [ ]:
InputFile_sr = "../data/ttH2015_forATLASit_ljets_Mbb_ge6jge4b_histos.root"

Set the data, set MC-stat uncertaintyy threshold to 5%

In [ ]:
chan_sr.SetData( "ljets_Mbb_ge6jge4b_Data", InputFile_sr, "ljets_Mbb_ge6jge4b/Data/nominal/" )
chan_sr.SetStatErrorConfig(0.05, 'Poisson')

Add signal sample, assing the POI as normalization factor to it

In [ ]:
signal_sr = ROOT.RooStats.HistFactory.Sample( "ttH", "ljets_Mbb_ge6jge4b_ttH", InputFile_sr, "ljets_Mbb_ge6jge4b/ttH/nominal/" )
signal_sr.AddNormFactor( "mu", 1, -10, 10 )

Add the background samples

In [ ]:
ttbar_sr = ROOT.RooStats.HistFactory.Sample( "ttbar", "ljets_Mbb_ge6jge4b_ttbar", InputFile_sr, "ljets_Mbb_ge6jge4b/ttbar/nominal/" )
stop_sr = ROOT.RooStats.HistFactory.Sample( "stop", "ljets_Mbb_ge6jge4b_singleTop", InputFile_sr, "ljets_Mbb_ge6jge4b/singleTop/nominal/" )

Assign a simple systematic to all the three samples (luminosity uncertainty, set to +/-5%)

In [ ]:
signal_sr.AddOverallSys( "lumi",  0.95, 1.05 )
ttbar_sr.AddOverallSys( "lumi",  0.95, 1.05 )
stop_sr.AddOverallSys( "lumi",  0.95, 1.05 )

Add the three samples to the channel

In [ ]:
chan_sr.AddSample( signal_sr )
chan_sr.AddSample( ttbar_sr )
chan_sr.AddSample( stop_sr )

Add channel to measurement

In [ ]:
meas.AddChannel( chan_sr )

Collect the histograms from their files, print some output, print the xml files

In [ ]:
meas.CollectHistograms()
meas.PrintTree()
meas.PrintXML('../xml', meas.GetOutputFilePrefix())

Create and save the workspace

In [ ]:
ROOT.RooStats.HistFactory.MakeModelAndMeasurementFast(meas)